In [2]:
import mlflow
mlflow.autolog()

2023/12/18 22:37:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [3]:
# @title Importation données
import pandas as pd
ct = 'QueryResults (1).csv'
df = pd.read_csv(ct)
df


,Id,Title,Body,Tags,Score,ViewCount,AnswerCount
0,20485075,How do I define multiple conditions on a join ...,"<p>Ok, lets say I have order and items tables....",<java><jpa><playframework><playframework-2.0><...,1,1984,2
1,20485079,What is the equivalent of a C# gridview in jav...,"<p>I am trying to learn java, I am working on ...",<c#><java><asp.net><swing><gridview>,2,12333,1
2,20485143,"Listview cursor adapter, always get first item...",<p>I have this piece of code for listview usin...,<android><listview><android-listview><simplecu...,1,1075,2
3,20485144,OpenSSL RSA-2048 unencrypted block is longer t...,<p>I am using the OpenSSL library in order to ...,<c><encryption><cryptography><openssl><rsa>,0,527,1
4,20485176,Returning char into main method,<p>The goal of my program is kind of like whee...,<java><eclipse><string><methods><char>,0,1278,3
...,...,...,...,...,...,...,...
49995,22814994,StructureMap use specific instance of type per...,"<p>Is there a way, using StructureMap (the Dep...",<c#><asp.net><dependency-injection><inversion-...,0,369,2
49996,22815009,Add a reference column migration in Rails 4,<p>A user has many uploads. I want to add a co...,<ruby-on-rails><ruby-on-rails-4><foreign-keys>...,368,397557,8
49997,22815010,Hibernate mapping with group by clause?,<p>I have <code>@OneToMany</code> mapping in a...,<java><sql><hibernate><group-by><hibernate-map...,0,1755,1
49998,22815022,java.lang.NoClassDefFoundError: android.suppor...,<p>I have a class that extends <code>android.s...,<java><android><eclipse><android-fragments><an...,0,1441,1


In [4]:
# @title Filtre des tags

df['Tags'] = df['Tags'].str.replace('><', ',').str.replace('<', '').str.replace('>', '')

from collections import Counter
# Fréquence des mots dans 'Tags'
all_tags = df['Tags'].str.split(',').explode().tolist()
tag_counts = Counter(all_tags)

# Trouver les 50 mots les plus fréquents
top_50_tags = [tag for tag, _ in tag_counts.most_common(50)]

# Filtrer les listes de mots pour ne contenir que les 50 mots les plus fréquents
def filter_tags(tags):
    return [tag for tag in tags if tag in top_50_tags]

df['Tags'] = df['Tags'].str.split(',').apply(filter_tags)

# Filtrer les lignes où 'Tags' est vide après le filtrage
df = df[df['Tags'].apply(len) > 0]

df['nb_tags'] = df['Tags'].apply(lambda x: len(x))
df = df[df['nb_tags']==3]

In [6]:
# @title Nettoyage du texte
import pickle
import nltk
nltk.download('Users')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

custom_punkt_path = '/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/corpora/punkt'
punkt_path = nltk.data.find(f'{custom_punkt_path}/english.pickle')
with open(punkt_path, 'rb') as file:
    punkt_model = pickle.load(file)
#nltk.data.path.append(custom_punkt_path)

with open('/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/corpora/stopwords/english', 'r') as file:
    custom_stopwords = file.read().splitlines()
    print('custom_stopwords existe')
    
# Nettoyage
def tokenizer_fct(sentence):
    print('entrée tokenizer')
    """Division de mots en texte + suppression de certains caractères"""
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    print('sentence cleaned')
    #word_tokens = word_tokenize(sentence_clean, language='english', path_to_punkt=punkt_path)
    word_tokens = punkt_model.tokenize(sentence_clean)
    print('tokenizer good')
    return word_tokens


stop_w = custom_stopwords + ['[', ']', ',', '.', ':', '?', '(', ')','<','>','~']
print('stop_w good')
def stop_word_filter_fct(list_words):
    print('fait')
    """Suppression de mots sans information+ ponctuations"""
    print('list_words type = ',type(list_words))
    filtered_w = [w for w in list_words if not w in stop_w]
    print('filtered_w good')
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    print('stop_word_filter_fct good')
    return filtered_w2

def lower_start_fct(list_words):
    print('un')
    """Conversion en lettres minuscules et suppression de préfixes indésirables"""
    lw = [w.lower() for w in list_words if (not w.startswith("@")) and (not w.startswith("#")) and (not w.startswith("http"))]
    print('lower good')
    return lw

def lemma_fct(list_words):
    print('truc')
    """lemmatisation"""
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    print('lemma good')
    return lem_w

def transform_bow_fct(desc_text):
    print('stp')
    """fonction de transformation"""
    word_tokens = tokenizer_fct(desc_text)
    print('word_tokens good')
    sw = stop_word_filter_fct(word_tokens)
    print('sw existe')
    lw = lower_start_fct(sw)
    print('lower  existe')
    transf_desc_text = ' '.join(lw)
    print('transform_bow_fct good')
    return transf_desc_text

# Prétraitement
df['Cleaned_Body'] = df['Body'].apply(transform_bow_fct)


[nltk_data] Error loading Users: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self signed certificate in certificate chain
[nltk_data]     (_ssl.c:997)>


custom_stopwords existe
stop_w good
stp
entrée tokenizer
sentence cleaned
tokenizer good
word_tokens good
fait
list_words type =  <class 'list'>
filtered_w good
stop_word_filter_fct good
sw existe
un
lower good
lower  existe
transform_bow_fct good
stp
entrée tokenizer
sentence cleaned
tokenizer good
word_tokens good
fait
list_words type =  <class 'list'>
filtered_w good
stop_word_filter_fct good
sw existe
un
lower good
lower  existe
transform_bow_fct good
stp
entrée tokenizer
sentence cleaned
tokenizer good
word_tokens good
fait
list_words type =  <class 'list'>
filtered_w good
stop_word_filter_fct good
sw existe
un
lower good
lower  existe
transform_bow_fct good
stp
entrée tokenizer
sentence cleaned
tokenizer good
word_tokens good
fait
list_words type =  <class 'list'>
filtered_w good
stop_word_filter_fct good
sw existe
un
lower good
lower  existe
transform_bow_fct good
stp
entrée tokenizer
sentence cleaned
tokenizer good
word_tokens good
fait
list_words type =  <class 'list'>
filtere

In [7]:
# @title Encoding TF - IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_array = tfidf_vectorizer.fit_transform(df['Cleaned_Body']).toarray()

# Liste des termes correspondant aux colonnes de la matrice
feature_names = tfidf_vectorizer.get_feature_names_out()
# Scores de chaque mot
tfidf_df = pd.DataFrame(tfidf_array, columns=feature_names)

# Concaténez les DataFrames sans utiliser de sample
df.reset_index(drop=True, inplace=True)
df2 = pd.concat([df, tfidf_df], axis=1)

Modèle supervisé

In [8]:
import random as rd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score


rd.seed(0)

def modelisation(enc,name):
    # Convertir les étiquettes ('Tags') en un format binaire
    mlb = MultiLabelBinarizer()
    #binary_labels = mlb.fit_transform(df['Tags']).tolist()
    # Diviser les données pour l'entraînement et le test
    X_train, X_test, y_train, y_test = train_test_split(enc, df['Tags'], test_size=0.2, random_state=42)
    y_train = [y[rd.randint(0, 2)] for y in y_train]
    y_test = [y[rd.randint(0, 2)] for y in y_test]

    # Initialisation et entrainement du modèle
    with mlflow.start_run(run_name= 'supervised_'+name+' last') as run:
        naive_bayes = MultinomialNB()
        naive_bayes.fit(X_train, y_train)
        mlflow.sklearn.log_model(
            sk_model=naive_bayes,
            artifact_path="model",
            registered_model_name="supervised_tfidf_vec",
        )
        mlflow.sklearn.log_model(
            sk_model=tfidf_vectorizer, 
            artifact_path="tfidf_vectorizer", 
            registered_model_name="supervised")


    # Prédictions
    predicted_tags = naive_bayes.predict(X_test)
    accuracy = accuracy_score(y_test, predicted_tags)

    print("La précision du modèle supervisé avec ",name," est de ",round(accuracy,2)*100," % ")

modelisation(tfidf_df,'tf_idf')

2023/12/18 22:37:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
2023/12/18 22:37:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/orphila_adjo

La précision du modèle supervisé avec  tf_idf  est de  19.0  % 


Evaluation du drift avec evidently

In [9]:
from evidently.test_suite import TestSuite
from evidently.test_preset import DataStabilityTestPreset

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

X_train, X_test, y_train, y_test = train_test_split(tfidf_df, df['Tags'], test_size=0.2, random_state=42)
y_train = [y[rd.randint(0, 2)] for y in y_train]
y_test = [y[rd.randint(0, 2)] for y in y_test]

df_y_train = pd.DataFrame(y_train, columns=['Target'])
df_y_test = pd.DataFrame(y_test, columns=['Target'])

data_stability= TestSuite(tests=[DataStabilityTestPreset(),])
data_stability.run(current_data=df_y_train, reference_data=df_y_test, column_mapping=None)
data_stability 

data_drift_report = Report(metrics=[DataDriftPreset(),])

data_drift_report.run(current_data=df_y_train, reference_data=df_y_test, column_mapping=None)
data_drift_report

In [13]:
data_stability.show()

In [14]:
data_drift_report.show()

In [9]:
# @title Extraction de caractéristiques avec Word2Vec
from gensim.models import Word2Vec
import numpy as np

sentences = [text.split() for text in df['Cleaned_Body']]

# Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Dictionnaire de mots vers vecteurs
word_vectors = word2vec_model.wv

# Obtention du vecteur d'un document en moyennant les vecteurs de ses mots
def document_vector(word_vectors, doc):
    # Supprimer les mots absents du vocabulaire du modèle
    doc = [word for word in doc if word in word_vectors.key_to_index]
    if len(doc) > 0:
        return np.mean(word_vectors[doc], axis=0)
    else:
        return np.zeros(word_vectors.vector_size)

word2vec_features = np.array([document_vector(word_vectors, doc) for doc in sentences])

word2vec_df = pd.DataFrame(word2vec_features, columns=[f'w2v_{i}' for i in range(word2vec_features.shape[1])])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
word2vec_df = scaler.fit_transform(word2vec_df)
modelisation(word2vec_df,'Word2Vec')

2023/12/14 21:11:01 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'list' object has no attribute 'flatten'
/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
Registered model 'supervised_tfidf_vec' already exists. Creating a new version of this model...
Created version '2' of model 'supervised_tfidf_vec'.
2023/12/14 21:11:14 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


La précision du modèle supervisé avec  Word2Vec  est de  10.0  % 


Registered model 'supervised' already exists. Creating a new version of this model...
Created version '12' of model 'supervised'.


In [ ]:
# @title Extraction de caractéristiques avec USE

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

# Modèle Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Obtention du vecteur d'un document
def get_use_embedding(text):
    # Convertir le texte en embeddings
    embeddings = embed([text])
    return np.array(embeddings).flatten()


use_features = np.array([get_use_embedding(text) for text in df['Cleaned_Body']])

use_df = pd.DataFrame(use_features, columns=[f'use_{i}' for i in range(use_features.shape[1])])
use_df = scaler.fit_transform(use_df)

modelisation(word2vec_df,'USE')

2023/11/24 21:19:35 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2023/11/24 21:20:30 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'list' object has no attribute 'flatten'
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


La précision du modèle supervisé avec  USE  est de  12.0  % 


Registered model 'supervised' already exists. Creating a new version of this model...
Created version '3' of model 'supervised'.


In [ ]:
# @title Extraction de caractéristiques avec BERT

from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np

# Modèle BERT et le tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Obtention du vecteur d'un document
def get_bert_embedding(text):
    # Tokenizer le texte et obtenir les embeddings
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    # Prendre la sortie correspondant au [CLS] token
    embeddings = outputs.last_hidden_state[:, 0, :].detach().numpy()
    return np.array(embeddings).flatten()


bert_features = np.array([get_bert_embedding(text) for text in df['Cleaned_Body']])

bert_df = pd.DataFrame(bert_features, columns=[f'bert_{i}' for i in range(bert_features.shape[1])])
bert_df = scaler.fit_transform(bert_df)

modelisation(bert_df,'BERT')

2023/11/24 21:20:59 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of transformers. If you encounter errors during autologging, try upgrading / downgrading transformers to a supported version, or try upgrading MLflow.
2023/11/24 21:21:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

2023/11/24 23:05:57 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'list' object has no attribute 'flatten'
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


La précision du modèle supervisé avec  BERT  est de  13.0  % 


Registered model 'supervised' already exists. Creating a new version of this model...
Created version '4' of model 'supervised'.
